# YOLOv11-seg training
使用 Linux(WSL) 環境訓練，用以支援多 GPU 訓練 (nccl)

YOLO 訓練資訊: https://docs.ultralytics.com/tasks/segment/#train

In [3]:
import comet_ml
import torch
from ultralytics import YOLO


## 檢查運算單元

In [4]:
check_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用裝置: {check_device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")

使用裝置: cuda
GPU: NVIDIA TITAN V
GPU 記憶體: 12.0 GB


## 視覺化訓練追蹤平台

[前往說明](https://docs.ultralytics.com/zh/modes/train/#augmentation-settings-and-hyperparameters:~:text=%E8%B5%84%E6%96%99%E9%83%A8%E5%88%86%E3%80%82-,%E8%AE%B0%E5%BD%95,-%E5%9C%A8%E8%AE%AD%E7%BB%83YOLO11)

透過 YOLO11 與 TensorBoard 的整合獲得視覺化洞察力 [前往](https://docs.ultralytics.com/zh/integrations/tensorboard/)

`Comet ml` 與 `ClearML` 擇一，不使用則 `pip uninstall clearml`

### Comet ml

In [ ]:
## Comet ML 是一個允許數據科學家和開發者追蹤、比較、解釋和優化實驗與模型的平台。它提供即時指標、程式碼差異和超參數追蹤等功能。
# https://docs.ultralytics.com/integrations/comet/#installation
# pip install comet_ml

# !export COMET_API_KEY=fxwf3W0gSNOYWuOd67xpC1LSR

# import comet_ml
# comet_ml.init() # set COMET_API_KEY
# comet_ml.login(project_name=project_folder, 
#                experiment_name=model_name,
#                workspace="felimet", 
#                api_key="fxwf3W0gSNOYWuOd67xpC1LSR")

COMET INFO: Valid Comet API Key saved in /home/isspmes/.comet.config (set COMET_CONFIG to change where it is saved).


### ClearML

In [ ]:
## ClearML 是一個開源平台，可自動追蹤實驗並協助高效共享資源。它旨在幫助團隊更有效地管理、執行和複製他們的機器學習工作。
# https://clear.ml/docs/latest/docs/clearml_sdk/clearml_sdk_setup
# pip install clearml
# import clearml

# clearml.browser_login()

### TensorBoard

In [ ]:
# TensorBoard 是一款視覺化工具包，適用於 TensorFlow。透過它，您可以將 TensorFlow 圖形視覺化，繪製有關圖形執行情況的量化指標，並顯示通過圖形的影像等附加資料。

# pip install tensorboard
# 在 Google Colab 中使用 TensorBoard：
# load_ext tensorboard
# tensorboard --logdir ultralytics/runs # replace with 'runs' directory

# 要在本地使用 TensorBoard，請執行以下命令並在以下位置查看結果 http://localhost:6006/。

## bash: 
# Enable TensorBoard logging
# yolo settings tensorboard=True

# 基本指令（讀取 ultralytics YOLO 訓練日誌）
# tensorboard --logdir training_result_yolo11m-seg/yolo11m-seg_cow 

# 指定連接埠（預設為 6006）
# tensorboard --logdir ultralytics/runs --port 8080

# 指定主機位址（允許遠端存取）
# tensorboard --logdir ultralytics/runs --host 0.0.0.0

# 重新載入間隔設定（秒）
# tensorboard --logdir ultralytics/runs --reload_interval 30

# 比較多個實驗目錄
# tensorboard --logdir name1:path1,name2:path2

# 設定最大載入檔案數
# tensorboard --logdir ultralytics/runs --max_reload_threads 4

# 除錯模式
# tensorboard --logdir ultralytics/runs --debugger_port 6064

## 導入 YOLO 基礎模型
自動創建資料 `base_model` 並下載 YOLO 基礎模型

In [ ]:
# Load the YOLO base model
base_model = "yolo11m-seg" # yolo11n-seg.pt yolo11s-seg.pt yolo11m-seg.pt yolo11l-seg.pt yolo11x-seg.pt
model = YOLO(f"../base_model/segmentation/{base_model}.pt")

project_folder = "seg_training_result"
model_name = f"{base_model}_cow"

# `Comet ml` 與 `ClearML` 擇一，不使用則 `pip uninstall clearml`
# Initialize Comet ML for tracking experiments
# import comet_ml
# comet_ml.init() # set COMET_API_KEY
comet_ml.login(project_name=project_folder, 
               experiment_name=model_name,
               workspace="felimet", 
               api_key="fxwf3W0gSNOYWuOd67xpC1LSR")

COMET INFO: Valid Comet API Key saved in /home/isspmes/.comet.config (set COMET_CONFIG to change where it is saved).


### 自訂 Comet ML 日誌
[Comet ML 官方文件](https://www.comet.com/docs/v2/integrations/third-party-tools/yolov8/)

`Comet ml` 與 `ClearML` 擇一，不使用則 `pip uninstall clearml`

In [6]:
import os

# 控制 Comet ML 在實驗過程中記錄的圖像預測數量。預設情況下，Comet ML 會記錄來自驗證集的 100 個圖像預測。
os.environ["COMET_MAX_IMAGE_PREDICTIONS"] = "200"

# 指定記錄批次圖像預測的頻率。記錄 COMET_EVAL_BATCH_LOGGING_INTERVAL 環境變數控制此頻率。預設值為 1。
os.environ["COMET_EVAL_BATCH_LOGGING_INTERVAL"] = "1"

# 通過將 COMET_EVAL_LOG_CONFUSION_MATRIX 環境變數設置為 "false" 來停用混淆矩陣日誌。混淆矩陣只會在訓練完成後記錄一次。
# os.environ["COMET_EVAL_LOG_CONFUSION_MATRIX"] = "false"

# 離線記錄訓練結果
# os.environ['COMET_START_ONLINE'] = '0'  



### TensorBoard 訓練時在終端機執行

In [ ]:
# 訓練時在終端機執行
# !tensorboard --logdir {project_fold}/{model_name} --port 7006 
# !tensorboard --logdir training_result_yolo11m-seg/yolo11m-seg_cow --port 7006 

## 訓練 YOLOv11-seg 
訓練完成的模型與結果儲存於 `training_result_yolo11m_seg` 資料夾中

In [ ]:
# import torch
# import gc

# 清空CUDA快取
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     torch.cuda.synchronize()

# # 強制垃圾回收
# gc.collect()

# import os
# os._exit(00)

/bin/bash: line 1: kill: (338926) - No such process


In [ ]:
# 訓練
# 詳細參數見 https://docs.ultralytics.com/tasks/segment/#train
# https://docs.ultralytics.com/zh/guides/yolo-data-augmentation/#example-configurations
# https://docs.ultralytics.com/zh/usage/cfg/#train-settings

# 設定訓練參數
results = model.train(
    data="../dataset_seg/yolo_data_integrated/dataset.yaml",

    # 訓練參數
    epochs=100,
    imgsz=512,
    device=[0, 1] if check_device == "cuda" else "cpu", 
    batch=32, 
    workers=8, 
    patience=25, 
    amp=True,

    # 訓練過程中使用的日誌記錄工具
    cache="disk",
    verbose=True,
    plots=True,
    save_period=10,
    save=True,  
    save_frames=True,
    save_json=True,
    pretrained=True,
    project=project_folder,
    name=model_name,
    exist_ok=True, # 允許覆蓋現有項目
    multi_scale=False, # 多尺度訓練
    optimizer="AdamW",  # "auto"
    seed=42,  # 設定隨機種子以確保可重現性

    # 學習率策略調整
    dropout=0.15,
    lr0=0.005, 
    lrf=0.0001,  
    warmup_epochs=3,  # 預熱階段
    warmup_momentum=0.0001, # 預熱階段動量
    warmup_bias_lr=0.005, # 預熱階段偏置學習率
    cos_lr=True,  # 使用餘弦學習率調度
    weight_decay=7e-4, # L2 正則化
    
    # 資料增強
    mosaic=0.8,    
    cutmix=0.5,
    mixup=0.5,
    copy_paste=0.3,
    # copy_paste_mode="mixup",  # copy-paste 模式

    # 幾何變換調整
    translate=0.15,
    scale=0.3
)

print("訓練完成！")
print(f"訓練結果儲存於: {project_folder}/{model_name}")

New https://pypi.org/project/ultralytics/8.3.156 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.154 🚀 Python-3.12.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN V, 12288MiB)
                                                        CUDA:1 (NVIDIA TITAN V, 12287MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=True, cutmix=0.5, data=./dataset_seg/yolo_data_integrated/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.15, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.5, mode=train, model

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: Experiment is live on comet.com https://www.comet.com/felimet/training-result-yolo11m-seg/66a507e71b8641f592599a07376e71c4



TensorBoard: Start with 'tensorboard --logdir training_result_yolo11m-seg/yolo11m-seg_cow', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1
Transferred 705/711 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 317.6±86.7 MB/s, size: 197.8 KB)


train: Scanning /home/isspmes/YOLOHub/dataset_seg/yolo_data_integrated/labels/train.cache... 3856 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3856/3856 [00:00<?, ?it/s]
train: Caching images (10.6GB Disk): 100%|██████████| 3856/3856 [00:00<00:00, 19048.78it/s]


val: Fast image access ✅ (ping: 1.2±0.7 ms, read: 146.8±99.5 MB/s, size: 175.3 KB)


...

                   all       1101       1858      0.992      0.994      0.995       0.99      0.992      0.994      0.995      0.839

100 epochs completed in 3.564 hours.
Optimizer stripped from training_result_yolo11m-seg/yolo11m-seg_cow/weights/last.pt, 45.1MB
Optimizer stripped from training_result_yolo11m-seg/yolo11m-seg_cow/weights/best.pt, 45.1MB

Validating training_result_yolo11m-seg/yolo11m-seg_cow/weights/best.pt...
Ultralytics 8.3.154 🚀 Python-3.12.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN V, 12288MiB)
                                                        CUDA:1 (NVIDIA TITAN V, 12287MiB)
YOLO11m-seg summary (fused): 138 layers, 22,336,083 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:27<00:00,  1.26it/s]


                   all       1101       1858      0.993      0.996      0.995       0.99      0.993      0.996      0.995       0.84
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 3.8ms postprocess per image
Saving training_result_yolo11m-seg/yolo11m-seg_cow/predictions.json...
Results saved to training_result_yolo11m-seg/yolo11m-seg_cow


COMET ERROR: We failed to read file 'training_result_yolo11m-seg/yolo11m-seg_cow/BoxF1_curve.png' for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file 'training_result_yolo11m-seg/yolo11m-seg_cow/BoxP_curve.png' for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file 'training_result_yolo11m-seg/yolo11m-seg_cow/BoxR_curve.png' for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file 'training_result_yolo11m-seg/yolo11m-seg_cow/BoxPR_curve.png' for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file 'training_result_yolo11m-seg/yolo11m-seg_cow/labels_correlogram.jpg' for uploading.
Please double-check the file path and permissions
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ----------------------------

訓練完成！
訓練結果儲存於: training_result_yolo11m-seg/yolo11m-seg_cow
